# **LP4_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets.

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs)."""

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

1. W_acres - Acres of land devoted to wheat.
2. C_acres - Acres of land devoted to corn.
3. S_acres - Acres of land devoted to sugar beets. 
4. W_produce - Tons of wheat produced. 
5. C_produce - Tons of corn produced. 
6. S_produce - Tons of sugar beets produced. 
7. W_bought - Tons of wheat bought from wholesaler.
8. C_bought - Tons of corn bought from wholesaler. 
9. W_sell - Tons of wheat sold. 
10. C_sell - Tons of corn sold. 
11. S_sell - Tons of sugar beets sold within quota.
12. S_excess - Tons of sugar beets sold in excess of quota.


In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response2.choices[0].message.content)

The objective function that describes this problem, where the goal is to minimize the total costs, will look like:

Minimize Z = 150*W_acres + 230*C_acres + 260*S_acres + 1.4*170*W_bought + 1.4*150*C_bought - 170*W_sell - 150*C_sell - 36*S_sell - 10*S_excess


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response3.choices[0].message.content)

Constraints:

1. Land allocation: 
   W_acres + C_acres + S_acres <= 500

2. Wheat production and use: 
   W_produce = 2.5 * W_acres
   W_bought + W_produce >= 200
   W_produce - W_bought = W_sell if W_produce > 200 else 0

3. Corn production and use: 
   C_produce = 3 * C_acres
   C_bought + C_produce >= 240
   C_produce - C_bought = C_sell if C_produce > 240 else 0

4. Sugar Beats Production and Selling:
   S_produce = 20 * S_acres
   S_produce <= S_sell + S_excess
   S_sell <= 6000
   S_excess = S_produce - S_sell if S_produce > 6000 else 0

5. Non-negativity: 
   The following variables should maintain the non-negativity constraints:
   W_acres, C_acres, S_acres, W_produce, C_produce, S_produce, W_bought, C_bought, W_sell, C_sell, S_sell, S_excess. 
   Therefore, they must be larger or equal to 0:

   W_acres, C_acres, S_acres, W_produce, C_produce, S_produce, W_bought, C_bought, W_sell, C_sell, S_sell, S_excess >= 0


### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response4.choices[0].message.content)

Here's a sample pyomo code for your problem:

```python
from pyomo.environ import *

model = ConcreteModel(name="Farm Optimization")

# Variables
model.wheat_acres = Var(domain=NonNegativeReals)
model.corn_acres = Var(domain=NonNegativeReals)
model.sugar_acres = Var(domain=NonNegativeReals)
model.wheat_bought = Var(domain=NonNegativeReals)
model.corn_bought = Var(domain=NonNegativeReals)

# Objective
model.profit = Objective(
    expr=(
        150*model.wheat_acres + 230*model.corn_acres + 260*model.sugar_acres 
        + 1.4*170*model.wheat_bought + 1.4*150*model.corn_bought 
        - 170*model.wheat_acres*2.5 - 150*model.corn_acres*3
        - 36*min(6000, model.sugar_acres*20) - 10*max(0, model.sugar_acres*20 - 6000)
    ),
    sense=minimize)

# Constraints
model.land = Constraint(expr=model.wheat_acres + model.corn_acres + model.sugar_acres <= 500)
model.wheat_balance = Constraint(expr=model.wheat_bought + model.wheat_acres*2.5 >= 200)
model.corn_balance = Constraint(expr=model.corn_bought + model.corn_acres*3 >= 240)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution
print("Profit = ", model.profit())
print("Wheat acres =", model.wheat_acres())
print("Corn acres = ", model.corn_acres())
print("Sugar beet acres = ", model.sugar_acres())
print("Wheat bought = ", model.wheat_bought())
print("Corn bought = ", model.corn_bought())
```

In this code, we created a minimization model to maximize the profit (which is negative in our case as we are minimizing costs) given constraints such as the land available, the production of wheat, corn, and sugar beets, and the selling quotas for each resource. We used the 'glpk' solver to solve the model and printed out the optimal solution.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

model = ConcreteModel(name="Farm Optimization")

# Variables
model.wheat_acres = Var(domain=NonNegativeReals)
model.corn_acres = Var(domain=NonNegativeReals)
model.sugar_acres = Var(domain=NonNegativeReals)
model.wheat_bought = Var(domain=NonNegativeReals)
model.corn_bought = Var(domain=NonNegativeReals)

# Objective
model.profit = Objective(
    expr=(
        150*model.wheat_acres + 230*model.corn_acres + 260*model.sugar_acres
        + 1.4*170*model.wheat_bought + 1.4*150*model.corn_bought
        - 170*model.wheat_acres*2.5 - 150*model.corn_acres*3
        - 36*min(6000, model.sugar_acres*20) - 10*max(0, model.sugar_acres*20 - 6000)
    ),
    sense=minimize)

# Constraints
model.land = Constraint(expr=model.wheat_acres + model.corn_acres + model.sugar_acres <= 500)
model.wheat_balance = Constraint(expr=model.wheat_bought + model.wheat_acres*2.5 >= 200)
model.corn_balance = Constraint(expr=model.corn_bought + model.corn_acres*3 >= 240)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution
print("Profit = ", model.profit())
print("Wheat acres =", model.wheat_acres())
print("Corn acres = ", model.corn_acres())
print("Sugar beet acres = ", model.sugar_acres())
print("Wheat bought = ", model.wheat_bought())
print("Corn bought = ", model.corn_bought())

PyomoException: Cannot convert non-constant Pyomo expression (20*sugar_acres  <  6000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *

model = ConcreteModel(name="Farm Optimization")

# Declare variables
model.W_acres = Var(domain=NonNegativeReals)
model.C_acres = Var(domain=NonNegativeReals)
model.S_acres = Var(domain=NonNegativeReals)
model.W_produce = Var(domain=NonNegativeReals)
model.C_produce = Var(domain=NonNegativeReals)
model.S_produce = Var(domain=NonNegativeReals)
model.W_bought = Var(domain=NonNegativeReals)
model.C_bought = Var(domain=NonNegativeReals)
model.W_sell = Var(domain=NonNegativeReals)
model.C_sell = Var(domain=NonNegativeReals)
model.S_sell = Var(domain=NonNegativeReals)
model.S_excess = Var(domain=NonNegativeReals)


model.profit = Objective(expr=
        150*model.W_acres + 230*model.C_acres + 260*model.S_acres
        + 1.4*170*model.W_bought + 1.4*150*model.C_bought
        - 170*model.W_sell - 150*model.C_sell
        - 36*model.S_sell - 10*model.S_excess,
    sense=minimize)

# Land allocation constraint
model.land_allocation = Constraint(expr=model.W_acres + model.C_acres + model.S_acres <= 500)

# Wheat production and use constraints
model.wheat_production = Constraint(expr=model.W_produce == 2.5 * model.W_acres)
model.wheat_usage = Constraint(expr=model.W_bought + model.W_produce >= 200)
model.wheat_selling = Constraint(expr=model.W_produce - model.W_bought == model.W_sell)

# Corn production and use constraints
model.corn_production = Constraint(expr=model.C_produce == 3 * model.C_acres)
model.corn_usage = Constraint(expr=model.C_bought + model.C_produce >= 240)
model.corn_selling = Constraint(expr=model.C_produce - model.C_bought == model.C_sell)

# Sugar Beets production and selling constraints
model.sugar_production = Constraint(expr=model.S_produce == 20 * model.S_acres)
model.sugar_selling_limit = Constraint(expr=model.S_sell <= 6000)
model.sugar_excess_calculation = Constraint(expr=model.S_excess == model.S_produce - model.S_sell)

# Adding additional constraints to handle the conditional selling logic
# Assume that wheat and corn selling only occurs if production exceeds certain thresholds
model.wheat_selling_condition = Constraint(expr=model.W_sell <= (model.W_produce - 200))
model.corn_selling_condition = Constraint(expr=model.C_sell <= (model.C_produce - 240))

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution
print("Profit = ", model.profit())


Profit =  -20600.0
